In [2]:
from bayleef import api
from datetime import date
import wget


In [3]:
api.login('krodriguez','Otacon1105')

{'errorCode': None,
 'error': '',
 'data': 'abcdc96a35a248e88715325f87023219',
 'api_version': '1.3.0',
 'access_level': 'guest',
 'executionTime': 0.53557300567627}

In [3]:
# get available metadata fields
api.dataset_fields('LANDSAT_8_C1', 'EE')

{'errorCode': None,
 'error': '',
 'data': [{'fieldId': 20520,
   'name': 'Landsat Product Identifier',
   'fieldLink': 'https://lta.cr.usgs.gov/Landsat_8_C1.html#landsat_product_id',
   'valueList': []},
  {'fieldId': 20514,
   'name': 'WRS Path',
   'fieldLink': 'https://lta.cr.usgs.gov/Landsat_8_C1.html#wrs_path',
   'valueList': []},
  {'fieldId': 20516,
   'name': 'WRS Row',
   'fieldLink': 'https://lta.cr.usgs.gov/Landsat_8_C1.html#wrs_row',
   'valueList': []},
  {'fieldId': 20522,
   'name': 'Land Cloud Cover',
   'fieldLink': 'https://lta.cr.usgs.gov/Landsat_8_C1.html#land_cloud_cover',
   'valueList': [{'value': None, 'name': 'All'},
    {'value': '10', 'name': 'Less than 10%'},
    {'value': '20', 'name': 'Less than 20%'},
    {'value': '30', 'name': 'Less than 30%'},
    {'value': '40', 'name': 'Less than 40%'},
    {'value': '50', 'name': 'Less than 50%'},
    {'value': '60', 'name': 'Less than 60%'},
    {'value': '70', 'name': 'Less than 70%'},
    {'value': '80', 'name'

In [66]:
%%time
start = date(year=2013, month=1, day=1)
end = date(year=2017, month=1, day=1)

# query for night time images around some Hawaiian Volcano
roi_images = api.search('LANDSAT_8_C1', 'EE', lat=19.414405, lng=-155.272403, distance=300,
           start_date='2013-04-01', end_date='2017-05-01', where={'20513':'Night'})

CPU times: user 27 ms, sys: 7.29 ms, total: 34.2 ms
Wall time: 4.65 s


In [74]:
roi_images['data']['numberReturned']

216

In [42]:
%%time
all_roi_images = api.search('LANDSAT_8_C1', 'EE', lat=19.414405, lng=-155.272403,
           start_date='2014-01-01', end_date='2015-01-01', where={'20513':'Night'})

CPU times: user 19.5 ms, sys: 3.29 ms, total: 22.8 ms
Wall time: 2.87 s


In [43]:
all_roi_images['data']['numberReturned']

66

In [44]:
all_roi_images['data']['results'][:1]

[{'acquisitionDate': '2014-12-31',
  'startTime': '2014-12-31',
  'endTime': '2014-12-31',
  'lowerLeftCoordinate': {'latitude': 19.83761, 'longitude': -153.74461},
  'upperLeftCoordinate': {'latitude': 18.10463, 'longitude': -153.35689},
  'upperRightCoordinate': {'latitude': 17.73773, 'longitude': -155.11101},
  'lowerRightCoordinate': {'latitude': 19.47255, 'longitude': -155.51769},
  'spatialFootprint': None,
  'sceneBounds': '-155.51769,17.73773,-153.35689,19.83761',
  'browseUrl': 'https://earthexplorer.usgs.gov/browse/landsat_8/2014/165/197/LC08_L1GT_165197_20141231_20170415_01_T2.jpg',
  'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&ordered=LC81651972014365LGN01&node=INVSVC',
  'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC81651972014365LGN01/INVSVC/',
  'entityId': 'LC81651972014365LGN01',
  'displayId': 'LC08_L1GT_165197_20141231_20170415_01_T2',
  'metadataUrl': 'https://earthexplorer.usgs.g

In [46]:
from datetime import datetime

datetime.strptime(all_roi_images['data']['results'][0]['acquisitionDate'], '%Y-%m-%d').year

2014

In [39]:
import re
import os

def get_path(response, root, dataset):
    if isinstance(response, dict):
        response = [response]
    
    try:
        for data in response:
            scene_id = data['entityId']
            s = data['summary']
            date = datetime.strptime(data['acquisitionDate'], '%Y-%m-%d')
            pathstr, rowstr = re.findall(r'Path: \b\d+\b|Row: \b\d+\b', s)
            path = pathstr.split(' ')[1]
            row = rowstr.split(' ')[1]
            return os.path.join(root, dataset, str(date.year), row, path, scene_id)
        
    except Exception as e:
        print('Failed to process request: {}'.format(e))
        

def bulk_download(response, root, dataset):
    results = response['data']['results']
    
    for scene in results:
        scene_id = scene['entityId']
        temp_file = '{}.tar.gz'.format(scene_id)
        
        path = get_path(scene, root, dataset)
        if os.path.exists(path):
            print('{} already in cache, skipping'.format(path))
            continue
        
        download_info = api.download('LANDSAT_8_C1', 'EE', scene_id)
        download_url = download_info['data'][0]
        
        print('Downloading: {} from {}'.format(scene_id, download_url))
        wget.download(download_url, temp_file)

        print('Extracting to {}'.format(path))
        tar = tarfile.open(temp_file) 
        tar.extractall(path=path)
        tar.close()
        print('Removing {}'.format(temp_file))
        os.remove(temp_file)
    
    print('done')
              
bulk_download(all_roi_images, '/data/sat_images', 'landsat8')

2016-12-27 00:00:00
Downloading: LC81661972016362LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/166/197/LC08_L1GT_166197_20161227_20170314_01_T2.tar.gz?id=o0lbpvadg8qao83as05346r4n1&iid=LC81661972016362LGN01&did=417281423&ver=production
Extracting to /data/sat_images/landsat8/2016/197/166/LC81661972016362LGN01
Removing LC81661972016362LGN01.tar.gz
2016-12-20 00:00:00
Downloading: LC81651972016355LGN02 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/197/LC08_L1GT_165197_20161220_20170315_01_T2.tar.gz?id=k6n718c7ilhug3htt5ol8c51g2&iid=LC81651972016355LGN02&did=417281740&ver=production
Extracting to /data/sat_images/landsat8/2016/197/165/LC81651972016355LGN02
Removing LC81651972016355LGN02.tar.gz
2016-12-20 00:00:00
Downloading: LC81651982016355LGN02 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/198/LC08_L1GT_165198_20161220_20170315_01_T2.tar.gz?id=j9vm8ovgh32arn4quei81tihn4&iid=LC8165

Extracting to /data/sat_images/landsat8/2016/197/166/LC81661972016250LGN01
Removing LC81661972016250LGN01.tar.gz
2016-08-30 00:00:00
Downloading: LC81651972016243LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/197/LC08_L1GT_165197_20160830_20170321_01_T2.tar.gz?id=4vdfhm2dbb0s0kh41osun7oa72&iid=LC81651972016243LGN01&did=417289148&ver=production
Extracting to /data/sat_images/landsat8/2016/197/165/LC81651972016243LGN01
Removing LC81651972016243LGN01.tar.gz
2016-08-30 00:00:00
Downloading: LC81651982016243LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/198/LC08_L1GT_165198_20160830_20170321_01_T2.tar.gz?id=nld5hsujc1n18rf479co9avld3&iid=LC81651982016243LGN01&did=417289429&ver=production
Extracting to /data/sat_images/landsat8/2016/198/165/LC81651982016243LGN01
Removing LC81651982016243LGN01.tar.gz
2016-08-21 00:00:00
Downloading: LC81661972016234LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/

Downloading: LC81651972016131LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/197/LC08_L1GT_165197_20160510_20170325_01_T2.tar.gz?id=6vfupdcdr62mc2oe1hqeep0jg6&iid=LC81651972016131LGN01&did=417296809&ver=production
Extracting to /data/sat_images/landsat8/2016/197/165/LC81651972016131LGN01
Removing LC81651972016131LGN01.tar.gz
2016-05-10 00:00:00
Downloading: LC81651982016131LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/198/LC08_L1GT_165198_20160510_20170325_01_T2.tar.gz?id=gurignddh2dga575aimt7894s5&iid=LC81651982016131LGN01&did=417297114&ver=production
Extracting to /data/sat_images/landsat8/2016/198/165/LC81651982016131LGN01
Removing LC81651982016131LGN01.tar.gz
2016-05-01 00:00:00
Downloading: LC81661972016122LGN01 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/166/197/LC08_L1GT_166197_20160501_20170325_01_T2.tar.gz?id=r0i6fldin65qn319id3u23e7p5&iid=LC81661972016122LGN01&did=

Extracting to /data/sat_images/landsat8/2016/197/165/LC81651972016019LGN02
Removing LC81651972016019LGN02.tar.gz
2016-01-19 00:00:00
Downloading: LC81651982016019LGN02 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/165/198/LC08_L1GT_165198_20160119_20170405_01_T2.tar.gz?id=p83nf6ep1sr8rqce8fcidom0k0&iid=LC81651982016019LGN02&did=417303777&ver=production
Extracting to /data/sat_images/landsat8/2016/198/165/LC81651982016019LGN02
Removing LC81651982016019LGN02.tar.gz
2016-01-10 00:00:00
Downloading: LC81661972016010LGN02 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T2/2016/166/197/LC08_L1GT_166197_20160110_20170405_01_T2.tar.gz?id=1rvhq1mf976luvqg2c48igg0h5&iid=LC81661972016010LGN02&did=417304070&ver=production
Extracting to /data/sat_images/landsat8/2016/197/166/LC81661972016010LGN02
Removing LC81661972016010LGN02.tar.gz
2016-01-03 00:00:00
Downloading: LC81651972016003LGN02 from https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/

In [5]:
response = api.metadata('LANDSAT_8_C1', 'EE', 'LC81651982016307LGN01')

In [23]:
import re 

In [12]:
response['data'][0]['orderUrl']

'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&ordered=LC81651982016307LGN01&node=INVSVC'

In [22]:
response['data'][0]

{'acquisitionDate': '2016-11-02',
 'startTime': '2016-11-02',
 'endTime': '2016-11-02',
 'lowerLeftCoordinate': {'latitude': 21.27999, 'longitude': -154.09952},
 'upperLeftCoordinate': {'latitude': 19.54772, 'longitude': -153.70743},
 'upperRightCoordinate': {'latitude': 19.17931, 'longitude': -155.4765},
 'lowerRightCoordinate': {'latitude': 20.91325, 'longitude': -155.88928},
 'spatialFootprint': None,
 'sceneBounds': '-155.88928,19.17931,-153.70743,21.27999',
 'browseUrl': 'https://earthexplorer.usgs.gov/browse/landsat_8/2016/165/198/LC08_L1GT_165198_20161102_20170318_01_T2.jpg',
 'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&ordered=LC81651982016307LGN01&node=INVSVC',
 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC81651982016307LGN01/INVSVC/',
 'entityId': 'LC81651982016307LGN01',
 'displayId': 'LC08_L1GT_165198_20161102_20170318_01_T2',
 'metadataUrl': 'https://earthexplorer.usgs.gov/metadata/xml/